In [ ]:
import matplotlib.pyplot as plt
from skimage import io, filters, util, restoration, morphology, exposure
import os
import numpy as np

In [ ]:
DATA_DIR = r'D:\Researchdata\ZY1'

In [ ]:
for parent, dir, file in os.walk(DATA_DIR):
    if( 'tiff' in parent):
        for f in file:
            raw_stack = io.imread(os.path.join(parent, f))
            break

In [ ]:
img_raw = raw_stack[2, ..., 2]
img_sharp = filters.unsharp_mask(img_raw, radius = 10, amount = 2, preserve_range = False)
img_sharp = util.img_as_uint(img_sharp)
plt.imshow(img_sharp, cmap = 'gray')

In [ ]:
background = restoration.rolling_ball(img_sharp, radius = 50)
plt.imshow(background, cmap = 'gray')

In [ ]:
background_normal = filters.rank.mean(background, selem = morphology.disk(30))
plt.imshow(background_normal, cmap = 'gray')

In [ ]:
img_bg_reduced = img_sharp - background_normal
img_bg_reduced[img_sharp < background_normal] = 0
# img_bg_reduced[img_bg_reduced <0 ] = 0
img_bg_reduced = exposure.rescale_intensity(img_bg_reduced, out_range = (np.min(img_sharp), np.max(img_sharp)))
# img_bg_reduced = (img_bg_reduced.astype(np.int16))
plt.imshow(img_bg_reduced, cmap = 'gray')
# plt.hist(img_bg_reduced.ravel(), bins=255)

In [ ]:
plt.hist(img_bg_reduced.ravel(), bins = 255)

In [ ]:
thresholds = filters.threshold_multiotsu(img_sharp, classes = 4)

In [ ]:

# Using the threshold values, we generate the three regions.
regions = np.digitize(img_bg_reduced, bins=thresholds)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 3.5))

# Plotting the original image.
ax[0].imshow(img_bg_reduced, cmap='gray')
ax[0].set_title('Original')
ax[0].axis('off')

# Plotting the histogram and the two thresholds obtained from
# multi-Otsu.
ax[1].hist(img_bg_reduced.ravel(), bins=255)
ax[1].set_title('Histogram')
for thresh in thresholds:
    ax[1].axvline(thresh, color='r')

# Plotting the Multi Otsu result.
ax[2].imshow(regions, cmap='jet')
ax[2].set_title('Multi-Otsu result')
ax[2].axis('off')

plt.subplots_adjust()

plt.show()
print(thresholds)
thresh = [thr for thr in thresholds if thr > 10][0]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.filters import threshold_multiotsu

# Setting the font size for all plots.
matplotlib.rcParams['font.size'] = 9

# The input image.
image = data.camera()

# Applying multi-Otsu threshold for the default value, generating
# three classes.
thresholds = threshold_multiotsu(image)

In [ ]:
temp = util.img_as_uint(img_bg_reduced)